In [1]:
with open('./sklearn_dp.txt','r') as f:
    sklearn_dps = f.read().split()

In [2]:
sklearn_dps

['sklearn.GridSearchCV',
 'sklearn.MLPClassifier',
 'sklearn.base.BaseEstimator.get_params',
 'sklearn.base.BaseEstimator.set_params',
 'sklearn.base.clone',
 'sklearn.base.clone.fit',
 'sklearn.calibration.CalibratedClassifierCV',
 'sklearn.calibration.calibration_curve',
 'sklearn.clone',
 'sklearn.cluster.AffinityPropagation',
 'sklearn.cluster.AgglomerativeClustering',
 'sklearn.cluster.Birch.fit',
 'sklearn.cluster.Birch.predict',
 'sklearn.cluster.DBSCAN',
 'sklearn.cluster.FeatureAgglomeration',
 'sklearn.cluster.KMeans',
 'sklearn.cluster.KMedoids',
 'sklearn.cluster.MeanShift',
 'sklearn.cluster.MiniBatchKMeans',
 'sklearn.cluster.SpectralClustering',
 'sklearn.cluster.Ward',
 'sklearn.cluster.affinity_propagation',
 'sklearn.cluster.bicluster.SpectralBiclustering',
 'sklearn.cluster.bicluster.SpectralCoclustering',
 'sklearn.cluster.estimate_bandwidth',
 'sklearn.cluster.hierarchical.AgglomerativeClustering',
 'sklearn.cluster.k_means',
 'sklearn.cluster.spectral_clustering',

In [3]:
sklearn_roots = ['.'.join(f.split('.')[:-1]) for f in sklearn_dps]

In [4]:
from importlib import import_module

In [8]:
sklearn_dp  = sklearn_dps

In [9]:
roots_counter = {}
for r in sklearn_roots:
    if r not in roots_counter:
        roots_counter[r]=0
    roots_counter[r]+=1
frequent_roots = {r: roots_counter[r] for r in roots_counter if roots_counter[r]>=5}
for r in frequent_roots:
#     if r=="statsmodels":continue
    try:
        mod = import_module(r)
#         print(mod.__all__)
        try:
            dps = ['{}.{}'.format(r, obj) for obj in mod.__all__] 
        except:
            dps =  ['{}.{}'.format(r, obj) for obj in dir(mod) if not obj.startswith('_')]
#         dps = ['{}.{}'.format(r, obj) for obj in dir(mod) if not obj.startswith('_')]
        sklearn_dp+=dps
#         print(dps)
    except Exception as e:
        print(e)
sklearn_dp = sorted(list(set(sklearn_dp)))

No module named 'sklearn.cross_validation'
No module named 'sklearn.model_selection.cross_val_score'


In [11]:
len(sklearn_dp)

750

In [14]:
funcs = [f for f in sklearn_dp if f.endswith("Regressor") or f.endswith("Classifier")]

In [15]:
funcs

['sklearn.MLPClassifier',
 'sklearn.dummy.DummyClassifier',
 'sklearn.dummy.DummyRegressor',
 'sklearn.ensemble.AdaBoostClassifier',
 'sklearn.ensemble.AdaBoostRegressor',
 'sklearn.ensemble.BaggingClassifier',
 'sklearn.ensemble.BaggingRegressor',
 'sklearn.ensemble.ExtraTreesClassifier',
 'sklearn.ensemble.ExtraTreesRegressor',
 'sklearn.ensemble.GradientBoostingClassifier',
 'sklearn.ensemble.GradientBoostingRegressor',
 'sklearn.ensemble.RandomForestClassifier',
 'sklearn.ensemble.RandomForestRegressor',
 'sklearn.ensemble.VotingClassifier',
 'sklearn.ensemble.VotingRegressor',
 'sklearn.ensemble.forest.RandomForestClassifier',
 'sklearn.ensemble.forest.RandomForestRegressor',
 'sklearn.ensemble.gradient_boosting.GradientBoostingClassifier',
 'sklearn.ensemble.gradient_boosting.GradientBoostingRegressor',
 'sklearn.gaussian_process.GaussianProcessClassifier',
 'sklearn.gaussian_process.GaussianProcessRegressor',
 'sklearn.linear_model.HuberRegressor',
 'sklearn.linear_model.Passive

In [23]:
target_funcs = funcs

In [18]:
nb_path = '/home/gezhang/data/jupyter/target'
notebooks = os.listdir(nb_path)

In [17]:
import os

In [19]:
sklearn_notebooks = []
for i, nb in enumerate(notebooks):
    if i%10000==0:
        print(i)
    with open(os.path.join(nb_path, nb),'r') as f:
#         print('--')
        content = f.read()
    tokens = content.split()
    if 'sklearn' in tokens:
        sklearn_notebooks.append(nb)
    

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000


In [20]:
len(sklearn_notebooks)

90345

In [21]:
sklearn_notebooks = sklearn_notebooks[:20000]

In [22]:
from extract_func import process_file

In [25]:
func2vector = {}
vector_size = len(sklearn_notebooks)
err_files = []

In [28]:
for f in target_funcs:
    func2vector[f] = np.zeros(vector_size)

In [27]:
import numpy as np

In [29]:
for i, nb in enumerate(sklearn_notebooks):
    if i%10000 == 0:
        print('Log: {} notebooks processed'.format(i))
    funcs = []
    try:
        funcs, linenos = process_file(os.path.join(nb_path, nb))
    except Exception as e:
        err_files.append(nb)
#     funcs = [func for func in funcs if func in no_loop_sklearn_funcs]
    for func in funcs:
        if func not in target_funcs:
            continue
#         if not func.startswith('sklearn'):
#             continue
#         if func not in func2vector:
#             func2vector[func] = np.zeros(vector_size)
        func2vector[func][i] = 1

Log: 0 notebooks processed
Log: 10000 notebooks processed


In [30]:
idx2func = list(func2vector.keys())
func2idx = {f: i for i, f in enumerate(idx2func)}
vectors = [func2vector[f] for f in idx2func]
occur_matrix = np.stack(vectors)
cooccur_matrix = np.dot(occur_matrix, occur_matrix.T)

In [32]:
cooccur_matrix.shape

(45, 45)

In [33]:
a = 1- cooccur_matrix/cooccur_matrix.max()

In [35]:
from sklearn import cluster


In [36]:
model = cluster.AgglomerativeClustering(n_clusters = None, distance_threshold=0.4, affinity="precomputed", linkage="average").fit(a)

In [37]:
clusters = []
n_samples = len(idx2func)
for child in model.children_:
    clusters.append([])
    for branch in child:
        if branch < n_samples:
            clusters[-1].append(idx2func[branch])
        else:
            clusters[-1]+=clusters[branch-n_samples]

# cooccurence

In [42]:
for c in clusters:
    if len(c)<=10:
#         print(c)
        print('-'*20)
#         print(c)
        for f in c:
            print(f)


--------------------
sklearn.ensemble.RandomForestClassifier
sklearn.tree.DecisionTreeClassifier
--------------------
sklearn.neighbors.KNeighborsClassifier
sklearn.ensemble.RandomForestClassifier
sklearn.tree.DecisionTreeClassifier
--------------------
sklearn.ensemble.AdaBoostClassifier
sklearn.neighbors.KNeighborsClassifier
sklearn.ensemble.RandomForestClassifier
sklearn.tree.DecisionTreeClassifier
--------------------
sklearn.ensemble.BaggingClassifier
sklearn.ensemble.AdaBoostClassifier
sklearn.neighbors.KNeighborsClassifier
sklearn.ensemble.RandomForestClassifier
sklearn.tree.DecisionTreeClassifier
--------------------
sklearn.gaussian_process.GaussianProcessClassifier
sklearn.ensemble.BaggingClassifier
sklearn.ensemble.AdaBoostClassifier
sklearn.neighbors.KNeighborsClassifier
sklearn.ensemble.RandomForestClassifier
sklearn.tree.DecisionTreeClassifier
--------------------
sklearn.ensemble.RandomForestRegressor
sklearn.tree.DecisionTreeRegressor
--------------------
sklearn.ensemb

In [49]:
s = 'aofA'
s[-1].isupper()
s+='f'

In [50]:
s

'aofAf'

In [55]:
import re


def split_func_name(func):
    new_str = ''
    for i, l in enumerate(func):
        if i>0 and l.isupper() and func[i-1].islower():
            new_str+='.'
        new_str+=l
    return re.split('\.|_', new_str)

In [56]:
split_func_name("sklearn.ensemble.AdaBoostClassifier")

['sklearn', 'ensemble', 'Ada', 'Boost', 'Classifier']

In [57]:
vocab = []
for f in target_funcs:
    tokens = split_func_name(f)
    for t in tokens:
        if t not in vocab:
            vocab.append(t)
        

In [58]:
len(vocab)

54

In [59]:
v2i = {v:i for i,v in enumerate(vocab)}

In [60]:
func2vector2 = {}
for f in target_funcs:
    func2vector2[f] = np.zeros(len(vocab))
    for t in split_func_name(f):
        func2vector2[f][v2i[t]]=1

In [61]:
func2vector2

{'sklearn.MLPClassifier': array([1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]),
 'sklearn.dummy.DummyClassifier': array([1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]),
 'sklearn.dummy.DummyRegressor': array([1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]),
 'sklearn.ensemble.AdaBoostClassifier': array([1., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [62]:
idx2func2 = list(func2vector2.keys())
func2idx2 = {f: i for i, f in enumerate(idx2func2)}
vectors2 = [func2vector2[f] for f in idx2func2]
occur_matrix2 = np.stack(vectors2)
cooccur_matrix2 = np.dot(occur_matrix2, occur_matrix2.T)

In [63]:
cooccur_matrix2

array([[2., 1., 1., ..., 1., 1., 1.],
       [1., 4., 3., ..., 1., 2., 1.],
       [1., 3., 4., ..., 2., 1., 2.],
       ...,
       [1., 1., 2., ..., 5., 3., 4.],
       [1., 2., 1., ..., 3., 5., 4.],
       [1., 1., 2., ..., 4., 4., 5.]])

In [66]:
a2 = 1- cooccur_matrix2/cooccur_matrix2.max()

In [67]:
a2

array([[0.71428571, 0.85714286, 0.85714286, ..., 0.85714286, 0.85714286,
        0.85714286],
       [0.85714286, 0.42857143, 0.57142857, ..., 0.85714286, 0.71428571,
        0.85714286],
       [0.85714286, 0.57142857, 0.42857143, ..., 0.71428571, 0.85714286,
        0.71428571],
       ...,
       [0.85714286, 0.85714286, 0.71428571, ..., 0.28571429, 0.57142857,
        0.42857143],
       [0.85714286, 0.71428571, 0.85714286, ..., 0.57142857, 0.28571429,
        0.42857143],
       [0.85714286, 0.85714286, 0.71428571, ..., 0.42857143, 0.42857143,
        0.28571429]])

In [68]:
model2 = cluster.AgglomerativeClustering(n_clusters = None, distance_threshold=0.4, affinity="precomputed", linkage="average").fit(a2)

In [69]:
clusters2 = []
n_samples2 = len(idx2func2)
for child in model2.children_:
    clusters2.append([])
    for branch in child:
        if branch < n_samples2:
            clusters2[-1].append(idx2func2[branch])
        else:
            clusters2[-1]+=clusters2[branch-n_samples2]

In [70]:
clusters2

[['sklearn.ensemble.gradient_boosting.GradientBoostingClassifier',
  'sklearn.ensemble.gradient_boosting.GradientBoostingRegressor'],
 ['sklearn.ensemble.RandomForestClassifier',
  'sklearn.ensemble.forest.RandomForestClassifier'],
 ['sklearn.ensemble.RandomForestRegressor',
  'sklearn.ensemble.forest.RandomForestRegressor'],
 ['sklearn.gaussian_process.GaussianProcessClassifier',
  'sklearn.gaussian_process.GaussianProcessRegressor'],
 ['sklearn.linear_model.PassiveAggressiveClassifier',
  'sklearn.linear_model.PassiveAggressiveRegressor'],
 ['sklearn.multiclass.OneVsOneClassifier',
  'sklearn.multiclass.OneVsRestClassifier'],
 ['sklearn.ensemble.GradientBoostingClassifier',
  'sklearn.ensemble.gradient_boosting.GradientBoostingClassifier',
  'sklearn.ensemble.gradient_boosting.GradientBoostingRegressor'],
 ['sklearn.ensemble.GradientBoostingRegressor',
  'sklearn.ensemble.GradientBoostingClassifier',
  'sklearn.ensemble.gradient_boosting.GradientBoostingClassifier',
  'sklearn.ensemb

# Common Sub Token

In [72]:
for c in clusters2:
    if len(c)<=15 and len(c)>3:
#         print(c)
        print('-'*20)
#         print(c)
        for f in c:
            print(f)

--------------------
sklearn.ensemble.GradientBoostingRegressor
sklearn.ensemble.GradientBoostingClassifier
sklearn.ensemble.gradient_boosting.GradientBoostingClassifier
sklearn.ensemble.gradient_boosting.GradientBoostingRegressor
--------------------
sklearn.ensemble.RandomForestClassifier
sklearn.ensemble.forest.RandomForestClassifier
sklearn.ensemble.RandomForestRegressor
sklearn.ensemble.forest.RandomForestRegressor
--------------------
sklearn.tree.DecisionTreeClassifier
sklearn.tree.ExtraTreeClassifier
sklearn.tree.DecisionTreeRegressor
sklearn.tree.ExtraTreeRegressor
--------------------
sklearn.linear_model.HuberRegressor
sklearn.linear_model.TheilSenRegressor
sklearn.linear_model.RidgeClassifier
sklearn.linear_model.PassiveAggressiveClassifier
sklearn.linear_model.PassiveAggressiveRegressor
--------------------
sklearn.linear_model.SGDRegressor
sklearn.linear_model.HuberRegressor
sklearn.linear_model.TheilSenRegressor
sklearn.linear_model.RidgeClassifier
sklearn.linear_model.P

In [74]:
occur_matrix2.shape

(45, 54)

In [73]:
occur_matrix.shape

(45, 20000)

In [78]:
m = np.append(occur_matrix, occur_matrix2, axis = 1)

In [79]:
model2 = cluster.AgglomerativeClustering(n_clusters = None, distance_threshold=0.4).fit(m)

In [80]:
clusters3 = []
n_samples2 = len(idx2func2)
for child in model2.children_:
    clusters3.append([])
    for branch in child:
        if branch < n_samples2:
            clusters3[-1].append(idx2func2[branch])
        else:
            clusters3[-1]+=clusters3[branch-n_samples2]

In [81]:
clusters3

[['sklearn.neighbors.RadiusNeighborsClassifier',
  'sklearn.neighbors.RadiusNeighborsRegressor'],
 ['sklearn.MLPClassifier', 'sklearn.ensemble.VotingRegressor'],
 ['sklearn.multioutput.MultiOutputClassifier',
  'sklearn.multioutput.MultiOutputRegressor'],
 ['sklearn.ensemble.gradient_boosting.GradientBoostingClassifier',
  'sklearn.ensemble.gradient_boosting.GradientBoostingRegressor'],
 ['sklearn.multiclass.OutputCodeClassifier',
  'sklearn.MLPClassifier',
  'sklearn.ensemble.VotingRegressor'],
 ['sklearn.multioutput.MultiOutputClassifier',
  'sklearn.multioutput.MultiOutputRegressor',
  'sklearn.multiclass.OutputCodeClassifier',
  'sklearn.MLPClassifier',
  'sklearn.ensemble.VotingRegressor'],
 ['sklearn.linear_model.HuberRegressor',
  'sklearn.linear_model.PassiveAggressiveRegressor'],
 ['sklearn.ensemble.forest.RandomForestClassifier',
  'sklearn.ensemble.forest.RandomForestRegressor'],
 ['sklearn.linear_model.stochastic_gradient.SGDClassifier',
  'sklearn.linear_model.HuberRegress

# sub token+ cooccur

In [82]:
for c in clusters3:
    if len(c)<=15 and len(c)>3:
#         print(c)
        print('-'*20)
#         print(c)
        for f in c:
            print(f)

--------------------
sklearn.multioutput.MultiOutputClassifier
sklearn.multioutput.MultiOutputRegressor
sklearn.multiclass.OutputCodeClassifier
sklearn.MLPClassifier
sklearn.ensemble.VotingRegressor
--------------------
sklearn.linear_model.TheilSenRegressor
sklearn.linear_model.stochastic_gradient.SGDClassifier
sklearn.linear_model.HuberRegressor
sklearn.linear_model.PassiveAggressiveRegressor
--------------------
sklearn.neighbors.RadiusNeighborsClassifier
sklearn.neighbors.RadiusNeighborsRegressor
sklearn.multioutput.MultiOutputClassifier
sklearn.multioutput.MultiOutputRegressor
sklearn.multiclass.OutputCodeClassifier
sklearn.MLPClassifier
sklearn.ensemble.VotingRegressor
--------------------
sklearn.ensemble.gradient_boosting.GradientBoostingClassifier
sklearn.ensemble.gradient_boosting.GradientBoostingRegressor
sklearn.ensemble.forest.RandomForestClassifier
sklearn.ensemble.forest.RandomForestRegressor
--------------------
sklearn.dummy.DummyRegressor
sklearn.neighbors.RadiusNeigh